# Pandas functions useful for formating and sanitizing

# Imports

In [49]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt


In [50]:
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 51)
pd.set_option('display.width', 1000)

# (Down)Load data

In [51]:
from loader import download_data
url_mtp = "https://data.montpellier3m.fr/sites/default/files/ressources/MMM_MMM_DAE.csv"


data = download_data(url_mtp)

# Using string dtype instead of object

Check ["Working with text data"](https://pandas.pydata.org/docs/user_guide/text.html)

In [52]:
data.nom.astype("string")

0      Plateau sportif de GrammontTerrain 9, 10, 11
1                 MEDIATHEQUE JEAN-JACQUES ROUSSEAU
2                   MEDIATHEQUE William SHAKESPEARE
3                                              MIBI
4                     MAISON POUR TOUS LEO LAGRANGE
                           ...                     
270                                  Zoo de Lunaret
271                              Gymnase Jean Bouin
272                        Centre Culturel Rabelais
273                                                
274                  Maison pour tous André Chamson
Name: nom, Length: 275, dtype: string

# Convert text to numeric
If you have numeric data strored as strings, you can convert them using [`pd.to_numeric`](https://pandas.pydata.org/docs/reference/api/pandas.to_numeric.html).
If you use `errors='coerce'`, strings that cannot be converted get replaced with `NaN`.

In [53]:
pd.to_numeric(data['lat_coor1'], errors='coerce').astype('float')

0      3.933921
1           NaN
2           NaN
3           NaN
4           NaN
         ...   
270    3.873468
271    3.820819
272    3.880320
273    3.869822
274    3.835223
Name: lat_coor1, Length: 275, dtype: float64

In [54]:
pd.to_numeric(data['long_coor1'], errors='coerce')

0      43.613635
1            NaN
2            NaN
3            NaN
4            NaN
         ...    
270    43.640944
271    43.632788
272    43.610690
273    43.612784
274    43.609273
Name: long_coor1, Length: 275, dtype: float64

In [55]:
pd.DataFrame(
    {'nom':pd.Series(['Noom A',
        'NOM B', 
        'Dept machin truc', 
        'Resto U',
        'Univ M', 
        'CPAM'],
        dtype='string')
    }
).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   nom     6 non-null      string
dtypes: string(1)
memory usage: 176.0 bytes


# Convert text to Datetime
Dates and times have a specific data type in pandas. You can convert strings to this type using [`pd.to_datetime`](https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html). 
If you use `errors='coerce'`, strings that cannot be converted get replaced with `NaT`.
Here is an example:

In [56]:
pd.to_datetime(
    data.dermnt, 
    errors='coerce',
    format='%Y-%m-%d')

0     2019-05-15
1            NaT
2            NaT
3            NaT
4     2020-10-06
         ...    
270   2019-05-14
271   2019-11-01
272          NaT
273          NaT
274   2019-12-01
Name: dermnt, Length: 275, dtype: datetime64[ns]

# Reformat a string using regular expressions

This example shows how to use regular expressions and the `str` functions  to reformat a string. We'll use phone numbers as an example:
This expression matches mutiple phone number formats and extracts figure pairs for formatting. 

In [57]:
extraction = data.tel1.str.extract(
        pat=r"\+*(?:33)0*(\d)\s*(\d\d)\s*(\d\d)\s*(\d\d)\s*(\d\d)",
        expand=True
    )
extraction

,0,1,2,3,4
0,4,67,64,87,70
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,4,67,40,33,57
...,...,...,...,...,...
270,4,67,54,45,20
271,4,67,75,44,43
272,4,67,34,71,33
273,NaN,NaN,NaN,NaN,NaN


A zero can be added to the begining of all numbers

In [58]:
'0'+ extraction[0]

0       04
1      NaN
2      NaN
3      NaN
4       04
      ... 
270     04
271     04
272     04
273    NaN
274     04
Name: 0, Length: 275, dtype: object

Figure pairs can be joined in a phone number using `str.cat` to concatenate:

In [59]:
"0" + extraction[0].str.cat(
    extraction[[i for i in range(1,5)]]
    )

0      0467648770
1             NaN
2             NaN
3             NaN
4      0467403357
          ...    
270    0467544520
271    0467754443
272    0467347133
273           NaN
274    0467751055
Name: 0, Length: 275, dtype: object

`str.cat` also supports including a separator:

In [60]:
"0" + extraction[0].str.cat(
    extraction[[1, 2, 3, 4]],
    sep='.'
    )

0      04.67.64.87.70
1                 NaN
2                 NaN
3                 NaN
4      04.67.40.33.57
            ...      
270    04.67.54.45.20
271    04.67.75.44.43
272    04.67.34.71.33
273               NaN
274    04.67.75.10.55
Name: 0, Length: 275, dtype: object

# Replace NA placeholders for pd.NA
In this example, missing phonenumbers marked with a dash get replaced by true NAs

In [61]:
have_dash = data.tel1 == '-'
data.tel1[have_dash]

45    -
69    -
70    -
Name: tel1, dtype: object

These do not get recognized as missing by pandas:

In [62]:
data.tel1[have_dash].isna()

45    False
69    False
70    False
Name: tel1, dtype: bool

You can mark them as missing using:

In [63]:
data.loc[have_dash,'tel1'] = pd.NA

data.tel1[have_dash]

45    <NA>
69    <NA>
70    <NA>
Name: tel1, dtype: object

In [64]:
data.tel1[have_dash].isna()

45    True
69    True
70    True
Name: tel1, dtype: bool

Note that an empty string is not considered na by pandas.

In [65]:
data.loc[have_dash, 'tel1'].fillna('').isna()

45    False
69    False
70    False
Name: tel1, dtype: bool